In [0]:
#getting all the links from the journal webpage in 'wiki' consisting all the volumes and issues and appending to href_list
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlparse, urljoin
href_list =[]
url = "https://link.springer.com"
wiki = "https://link.springer.com/journal/42822/volumes-and-issues"
href_list = []
#Query the website and return the html to the variable 'page'
page = urllib.request.urlopen(wiki)
soup = BeautifulSoup(page) 
for link in soup.find_all("a", class_="u-interface-link u-text-sans-serif u-text-sm"):
  href = link['href']
  href = urljoin(url, href)
  href_list.append(href)


In [0]:
#checking whether for a link,next page exists or not if exists append that link also to href_list
def no_of_pages(href_list):
  set1  = set()
  for i in href_list:
    page = urllib.request.urlopen(i)
    soup = BeautifulSoup(page) 
    for link in soup.find_all("a", class_="next"):
      hreff = link['href']
      hreff2 = url + hreff
      set1.add(hreff2)
  return set1

list2 = []
set1 = no_of_pages(href_list) 
while(len(set1)>0):
  for value in set1:
    list2.append(value)
  set1 = no_of_pages(list2)
for i in list2:
  href_list.append(i)

In [0]:
href_list

['https://link.springer.com/journal/40614/42/4',
 'https://link.springer.com/journal/40614/42/3',
 'https://link.springer.com/journal/40614/42/2',
 'https://link.springer.com/journal/40614/42/1',
 'https://link.springer.com/journal/40614/41/2',
 'https://link.springer.com/journal/40614/41/1',
 'https://link.springer.com/journal/40614/40/2',
 'https://link.springer.com/journal/40614/40/1',
 'https://link.springer.com/journal/40614/39/2',
 'https://link.springer.com/journal/40614/39/1',
 'https://link.springer.com/journal/40614/38/2',
 'https://link.springer.com/journal/40614/38/1',
 'https://link.springer.com/journal/40614/37/2',
 'https://link.springer.com/journal/40614/37/1',
 'https://link.springer.com/journal/40614/36/2',
 'https://link.springer.com/journal/40614/36/1',
 'https://link.springer.com/journal/40614/35/2',
 'https://link.springer.com/journal/40614/35/1',
 'https://link.springer.com/journal/40614/34/2',
 'https://link.springer.com/journal/40614/34/1',
 'https://link.sprin

In [0]:
#Now getting the article links from the journal links and appending all article links to href_list1
import urllib.request
from bs4 import BeautifulSoup
import re
import pandas as pd
from urllib.parse import urlparse, urljoin
url = "https://link.springer.com"
href_list1 = []
href1 = ''
k = 0 
#Query the website and return the html to the variable 'page'
for link in href_list:
  k  = k + 1
  page = urllib.request.urlopen(link)
  soup = BeautifulSoup(page) 
  list = []
  class1 = 'toc-item no-access'
  class2 = 'toc-item'
  
  selects =soup.findAll(class_='webtrekk-track pdf-link')
  selects1 =soup.findAll(class_ = 'webtrekk-track fulltext')
  for match in selects:
    match.decompose()
  for match in selects1:
    match.decompose()
  
  for x in soup.findAll("div", attrs = {'class' : [class1,class2]}):
    links = x.findAll('a')
    for i in links:
      if 'article' in i['href']:
        if 'pdf' not in i['href']:
          href1 = urljoin(url, i['href'])
          href_list1.append(href1)
        

In [0]:
#Now getting all the references from the article links and appending to the list2 
import xlsxwriter
from xlsxwriter import Workbook
import os, glob,xlwt
import time
from socket import error as SocketError
import errno
import re
wbk = Workbook('journal8.xlsx')
sheet = wbk.add_worksheet('data')
list2 = []
count = 0
comb = ''
authorandYear = ''
row = 0
clean = re.compile('<.*?>')
for url in href_list1:
  try:
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page)
    seven_day=soup.find('div', {'id' : 'Bib1-section'})
  except SocketError as e:
    if e.errno != errno.ECONNRESET:
        raise # Not error we are looking for
    else:
      pass # Handle error here.
  if seven_day == None:
    pass
  else:
    forecast_items = seven_day.find_all(class_="c-article-references__text")
    #print(forecast_items)
    for i in forecast_items:
      j = (i.contents)
      count = 0
      for k in range(0,len(j)):
        if 'retreived' in str(k):
          count+=1 
      if count > 0:
        for i in j:
          i = re.sub(r'<.*?>', '', str(i),flags=re.MULTILINE)
          i = re.sub(r'^https?:\/\/.*[\r\n]*', '', str(i), flags=re.MULTILINE)
          comb = comb + i
      else:
        i = re.sub(r'<.*?>', '', str(i),flags=re.MULTILINE)
        comb = comb + i
      #write or edit the cleaned reference below
      comb1 = re.sub(r'^.*[(]\d{4}.*?[)]\.?','',str(comb))
      journalandarticle = comb1.split('.',1)
      #print(journalandarticle)
      article = journalandarticle[0]
      if len(journalandarticle) != 2 :
        journal = ''
      else:
        journal = journalandarticle[1]
        journal = re.sub(r',\s?\d.*','' ,journal)
      authorandYear = re.findall(r'.*[(]\d{4}.*?[)]',str(comb))
      year = re.findall(r'\d{4}',str(authorandYear))
      if len(year)<1:
        year = ''
      else:
        year = year[0]
      if len(authorandYear)<1:
        authorandYear = ''
      else:
        authorandYear = authorandYear[0]
      author = re.sub(r'[(]\d{4}.*?[)]','',authorandYear)
      #print(author)
      sheet.write(row,0,str(journal))
      sheet.write(row,1,str(article))
      sheet.write(row,2,str(year))
      sheet.write(row,3,str(author))      
      row+=1
      comb = ''
      print(row)
wbk.close()